In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,roc_auc_score,plot_roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/kaggle/input/employee-attrition/HR-Employee-Attrition.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.corr()

In [ ]:
lst=[]
for i in df.columns:
    lst.append(i)
print(lst)
lst.pop(1)
lst
    

In [ ]:
for i in lst:
    plt.figure(figsize=(16,8))
    sns.countplot(df[i],hue=df['Attrition'])    
    plt.show()

In [ ]:
df.info()

In [ ]:
df['Age'].max()

In [ ]:
lst1=[]
lst2=[]
lst3=[]
lst4=[]
lst5=[]
for i in df['Age']:
    if i>=18 and i <28:
        lst1.append(i)
    elif i>=28 and i<36:
        lst2.append(i)
    elif i>=36 and i<42:
        lst3.append(i)
    elif i>=42 and i<50:
        lst4.append(i)
    else:
        lst5.append(i)
lst1=list(set(lst1))
lst2=list(set(lst2))
lst3=list(set(lst3))
lst4=list(set(lst4))
lst5=list(set(lst5))
df['Age']=df['Age'].replace(lst1,'18-28')
df['Age']=df['Age'].replace(lst2,'28-36')
df['Age']=df['Age'].replace(lst3,'36-42')
df['Age']=df['Age'].replace(lst4,'42-50')
df['Age']=df['Age'].replace(lst5,'>50')

In [ ]:
df['Age'].unique()


In [ ]:
df['Attrition'].value_counts()

In [ ]:
df['Attrition'].unique()

In [ ]:
print(df['BusinessTravel'].value_counts())
print(df['BusinessTravel'].unique())

In [ ]:
for i in df.columns:
    if df[i].dtype !='int64':
        print(i)

In [ ]:
print(df['Department'].unique())
print(df['EducationField'].unique())

In [ ]:
dict1={'Human Resources':'Other'}
df['EducationField']=df['EducationField'].replace('Human Resources','Other')

In [ ]:
df['EducationField'].value_counts()


In [ ]:
print(df['Gender'].unique())
print(df['Gender'].value_counts())

In [ ]:
print(df['JobRole'].unique())
print(df['JobRole'].value_counts())

In [ ]:
df['JobRole']=df['JobRole'].replace('Sales Representative','Sales Executive')

In [ ]:
df.groupby(['JobRole','Attrition']).size()

In [ ]:
df['MaritalStatus'].unique()

In [ ]:
del df['Over18']

In [ ]:
df.groupby(['OverTime','Attrition']).Attrition.count()

In [ ]:
numerical=df.select_dtypes(include=np.number).columns

In [ ]:
numerical

In [ ]:
df['DistanceFromHome'].unique()
df['Education'].unique()
df.groupby(['Education','Attrition']).size()

In [ ]:
del df['EmployeeCount']

In [ ]:

df.groupby(['EnvironmentSatisfaction','Attrition']).size()

In [ ]:
df['JobInvolvement'].unique()
df.groupby(['JobInvolvement','Attrition']).size()

In [ ]:
df['JobLevel'].unique()

In [ ]:

df['JobSatisfaction'].unique()

In [ ]:
df['NumCompaniesWorked'].unique()
df['PercentSalaryHike'].unique()
df['PerformanceRating'].unique()
df['RelationshipSatisfaction'].unique()

In [ ]:
df.groupby(['RelationshipSatisfaction','Attrition']).size()
df['StandardHours'].unique()

In [ ]:
del df['StandardHours']

In [ ]:
df['StockOptionLevel'].unique()
df['StockOptionLevel'].value_counts()
df['TrainingTimesLastYear'].unique()
df['WorkLifeBalance'].unique()
df['YearsAtCompany'].unique()

In [ ]:
lb=LabelEncoder()
df['PerformanceRating']=lb.fit_transform(df['PerformanceRating'])

In [ ]:
categorical=df.select_dtypes(exclude=np.number).columns

In [ ]:
categorical

In [ ]:
df['OverTime'].unique()

In [ ]:
cat1=pd.get_dummies(df['Age'],prefix='Age',drop_first=True)
cat2=pd.get_dummies(df['BusinessTravel'],prefix='BusinessTravel',drop_first=True)
cat3=pd.get_dummies(df['Department'],prefix='Department',drop_first=True)
cat4=pd.get_dummies(df['EducationField'],prefix='EducationField',drop_first=True)
cat5=pd.get_dummies(df['Gender'],prefix='Gender',drop_first=True)
cat6=pd.get_dummies(df['JobRole'],prefix='JobRole',drop_first=True)
cat7=pd.get_dummies(df['MaritalStatus'],prefix='MaritalStatus',drop_first=True)
cat8=pd.get_dummies(df['OverTime'],prefix='OverTime',drop_first=True)

In [ ]:
df.select_dtypes(include=np.number).columns

In [ ]:
df_final=pd.concat([cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,
           df[['DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager','Attrition']]],axis=1)

In [ ]:
df_final.shape

In [ ]:
numerical1=df.select_dtypes(include=np.number)

In [ ]:
vif=pd.DataFrame()
vif['Feature']=numerical1.columns
vif['Data']=[variance_inflation_factor(numerical1.values,i) for i in range(len(numerical1.columns))]

In [ ]:
vif

In [ ]:
df_final.columns

In [ ]:
df_final['Attrition']=np.where(df_final['Attrition']=='Yes',1,0)

In [ ]:
X= df_final[['Age_28-36', 'Age_36-42', 'Age_42-50', 'Age_>50',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Research & Development', 'Department_Sales',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'Gender_Male', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 'JobRole_Sales Executive',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'OverTime_Yes',
       'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager']]
y=df_final['Attrition']


In [ ]:
train_x,test_x,train_y,test_y= train_test_split(X,y,test_size=0.3,random_state=123)

## Logistic Regression

In [ ]:
sm=StandardScaler()
train_scaled=sm.fit_transform(train_x)
test_scaled=sm.fit_transform(test_x)

In [ ]:
lm=LogisticRegression()

In [ ]:
lm.fit(train_scaled,train_y)
pred_train=lm.predict(train_scaled)
pred_test=lm.predict(test_scaled)

In [ ]:
def metric_score(actual, pred):
    print('confusion matrix: ', confusion_matrix(actual, pred))
    print('accuracy score: ', accuracy_score(actual, pred))
    print('recall_score: ', recall_score(actual, pred))
    print('precision score: ', precision_score(actual, pred))
    print('f1 score: ', f1_score(actual, pred))
    print('roc auc score: ', roc_auc_score(actual, pred))
    print('classification report : \n', classification_report(actual, pred))

In [ ]:
metric_score(train_y,pred_train)

In [ ]:
metric_score(test_y,pred_test)

## KNN

In [ ]:
def KNN_predicts(train_x, test_x, train_y,test_y, scaler, neighbours, 
                 metric = 'manhattan', 
                 weights = 'uniform'):

    train_scaled = scaler.fit_transform(train_x)
    test_scaled = scaler.transform(test_x)

    KNN = KNeighborsClassifier(n_neighbors= neighbours, metric = metric, 
                               weights = weights, n_jobs = -1)
    KNN.fit(train_scaled, train_y)
    prediction_train = KNN.predict(train_scaled)
    prediction_test = KNN.predict(test_scaled)

    print('Accuracy Train: ', accuracy_score(train_y, prediction_train))
    print('Accuracy Test: ', accuracy_score(test_y, prediction_test))
    print('-'*100)
    return KNN

In [ ]:
KNN_predicts(train_x, test_x, train_y,test_y, StandardScaler(),1)

In [ ]:
for k in range(1, 11):
    print('Accuracy score on KNN using n_neighbors = {0}:'.format(k), end = ' ')
    KNN_predicts(train_x, test_x, train_y,test_y,StandardScaler(), k)

In [ ]:
k=5
for metric in ['euclidean','minkowski','manhattan','chebyshev']:
    print('Accuracy score on KNN using {} metric and {} neighbors:'.format(metric,k), end = ' ')
    KNN_predicts(train_x, test_x, train_y,test_y, StandardScaler(), k,metric)

In [ ]:
for weight in ['uniform','distance']:
    print('Accuracy score on KNN using  neighbors {} weights: '.format(weight), end = ' ')
    KNN_predicts(train_x, test_x, train_y,test_y, StandardScaler(), 5,'minkowski',weights=weight)

In [ ]:
plot_roc_curve(lm,train_scaled,train_y)

In [ ]:
plot_roc_curve(lm,test_scaled,test_y)

## Handling imbalanced Dataset

In [ ]:
import imblearn
print(imblearn.__version__)
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
counter = Counter(train_y)
print(counter)

In [ ]:
counter = Counter(test_y)
print(counter)

In [ ]:
from imblearn.combine import SMOTETomek
ss=StandardScaler()
scaled_X=ss.fit_transform(X)
sm = SMOTETomek(0.5, n_jobs=-1)
X_Train, y_train = sm.fit_resample(scaled_X, y)
train_x,test_x,train_y,test_y= train_test_split(X_Train,y_train,test_size=0.3,random_state=123)


print(f'The Number of class before the fit \n {train_y.value_counts()}')

print(f'The Number of class after the fit \n {y_train.value_counts()}')

In [ ]:
lm.fit(train_x,train_y)
pred_train=lm.predict(train_x)
pred_test=lm.predict(test_x)

In [ ]:
def metric_score(actual, pred):
    print('confusion matrix: ', confusion_matrix(actual, pred))
    print('accuracy score: ', accuracy_score(actual, pred))
    print('recall_score: ', recall_score(actual, pred))
    print('precision score: ', precision_score(actual, pred))
    print('f1 score: ', f1_score(actual, pred))
    print('roc auc score: ', roc_auc_score(actual, pred))
    print('classification report : \n', classification_report(actual, pred))

In [ ]:
metric_score(train_y,pred_train)

In [ ]:
plot_roc_curve(lm,train_x,train_y)

In [ ]:
metric_score(test_y,pred_test)

In [ ]:
plot_roc_curve(lm,test_x,test_y)

## KNN

In [ ]:
def KNN_predict(X,y, neighbours, 
                 metric = 'manhattan', 
                 weights = 'uniform'):
    ss=StandardScaler()
    scaled_X=ss.fit_transform(X)
    sm = SMOTETomek(0.5, n_jobs=-1)
    X_Train, y_train = sm.fit_resample(scaled_X, y)
    train_x,test_x,train_y,test_y= train_test_split(X_Train,y_train,test_size=0.3,random_state=123)

    KNN = KNeighborsClassifier(n_neighbors= neighbours, metric = metric, 
                               weights = weights, n_jobs = -1)
    KNN.fit(train_x, train_y)
    prediction_train = KNN.predict(train_x)
    prediction_test = KNN.predict(test_x)

    print('Accuracy Train: ', accuracy_score(train_y, prediction_train))
    print('Accuracy Test: ', accuracy_score(test_y, prediction_test))
    print('-'*100)
    return KNN

In [ ]:
KNN_predict(X,y,neighbours=1)

In [ ]:
for k in range(1, 11):
    print('Accuracy score on KNN using n_neighbors = {0}:'.format(k), end = ' ')
    KNN_predict(X,y, neighbours=k)

In [ ]:
k=6
for metric in ['euclidean','minkowski','manhattan','chebyshev']:
    print('Accuracy score on KNN using {} metric and {} neighbors:'.format(metric,k), end = ' ')
    KNN_predict(X,y, k,metric)

In [ ]:
for weight in ['uniform','distance']:
    print('Accuracy score on KNN using  neighbors {} weights: '.format(weight), end = ' ')
    KNN_predict(X,y, 6,'manhattan',weights=weight)

In [ ]:
ss=StandardScaler()
scaled_X=ss.fit_transform(X)
sm = SMOTETomek(0.5, n_jobs=-1)
X_Train, y_train = sm.fit_resample(scaled_X, y)
train_x,test_x,train_y,test_y= train_test_split(X_Train,y_train,test_size=0.3,random_state=123)

KNN = KNeighborsClassifier(n_neighbors= 6, metric = 'manhattan', 
                           weights = 'uniform', n_jobs = -1)
KNN.fit(train_x, train_y)
prediction_train = KNN.predict(train_x)
prediction_test = KNN.predict(test_x)

In [ ]:
metric_score(train_y,prediction_train)

In [ ]:
metric_score(test_y,prediction_test)

In [ ]:
plot_roc_curve(KNN,train_x,train_y)

In [ ]:
plot_roc_curve(KNN,test_x,test_y)

## Guasian NB

In [ ]:
gn= GaussianNB()

In [ ]:
gn.fit(train_x, train_y)
train_pred=gn.predict(train_x)
test_pred=gn.predict(test_x)

In [ ]:
metric_score(train_y,train_pred)

In [ ]:
metric_score(test_y,test_pred)

## Conclusion 
By handling the Unbiased data with SMOTE analysis and Logistic Regression/KNN gives better result